In [1]:
%%local
import os
import json
from IPython import get_ipython

username = os.environ['RENKU_USERNAME']

configuration = dict(
    name = f"{username}-final-project",
    executorMemory = "4G",
    executorCores = 4,
    numExecutors = 10,
    conf = {
        
        "spark.jars.repositories": "https://repos.spark-packages.org",
        "spark.jars.packages": "graphframes:graphframes:0.8.2-spark2.4-s_2.11"
    }
)


get_ipython().run_cell_magic('configure', line="-f", 
                             cell=json.dumps(configuration))

ID,YARN Application ID,Kind,State,Spark UI,Driver log,User,Current session?
2876,application_1680948035106_2698,pyspark,idle,Link,Link,None,
2877,application_1680948035106_2699,pyspark,idle,Link,Link,None,
2878,application_1680948035106_2700,pyspark,idle,Link,Link,None,
2880,application_1680948035106_2701,pyspark,idle,Link,Link,None,


In [2]:
%spark

Starting Spark application


ID,YARN Application ID,Kind,State,Spark UI,Driver log,User,Current session?
2881,application_1680948035106_2702,pyspark,idle,Link,Link,None,✔


FloatProgress(value=0.0, bar_style='info', description='Progress:', layout=Layout(height='25px', width='50%'),…

SparkSession available as 'spark'.


FloatProgress(value=0.0, bar_style='info', description='Progress:', layout=Layout(height='25px', width='50%'),…

In [3]:
%%send_to_spark -i username -t str -n username

FloatProgress(value=0.0, bar_style='info', description='Progress:', layout=Layout(height='25px', width='50%'),…

Successfully passed 'username' as 'username' to Spark kernel

In [4]:
print('We are using Spark %s' % spark.version)

FloatProgress(value=0.0, bar_style='info', description='Progress:', layout=Layout(height='25px', width='50%'),…

We are using Spark 2.4.8.7.1.8.0-801

#Load data

First, run "hdfs dfs -ls /data/sbb/part_orc/timetables" in the terminal and then 

In [24]:
orc_file_path = "/data/sbb/part_orc/timetables"

stops = spark.read.orc(orc_file_path + "/stops")
stop_times = spark.read.orc(orc_file_path + "/stop_times")
calendar = spark.read.orc(orc_file_path + "/calendar")
routes = spark.read.orc(orc_file_path + "/routes")
trips = spark.read.orc(orc_file_path + "/trips")

FloatProgress(value=0.0, bar_style='info', description='Progress:', layout=Layout(height='25px', width='50%'),…

In [26]:
csv_file_path = "/data/sbb/part_csv/timetables"
stops_csv = spark.read.csv(csv_file_path + "/stops", header = True)

FloatProgress(value=0.0, bar_style='info', description='Progress:', layout=Layout(height='25px', width='50%'),…

In [27]:
stops_csv.show()

FloatProgress(value=0.0, bar_style='info', description='Progress:', layout=Layout(height='25px', width='50%'),…

+-------+--------------------+----------------+----------------+-------------+--------------+----+-----+---+
|stop_id|           stop_name|        stop_lat|        stop_lon|location_type|parent_station|year|month|day|
+-------+--------------------+----------------+----------------+-------------+--------------+----+-----+---+
|1100008|Zell (Wiesental),...|47.7100842702352|7.85964788274668|         null|          null|2022|   12|  7|
|1100009|Zell (Wiesental),...|47.7131911044794|7.86290876722849|         null|          null|2022|   12|  7|
|1100010|           Atzenbach|47.7146175266411| 7.8723500608659|         null|          null|2022|   12|  7|
|1100011|     Mambach, Brücke|47.7282088873189| 7.8774704579861|         null|          null|2022|   12|  7|
|1100012|  Mambach, Mühlschau|47.7340818684375| 7.8813871126254|         null|          null|2022|   12|  7|
|1100013|  Mambach, Silbersau|47.7395192233867|7.88223152899259|         null|          null|2022|   12|  7|
|1100014|Fröhnd (Sc

In [31]:
stops.show() #We can see the orc_file is not precise in showing stop_lat and stop_lon, so we chose the csv files. Other files are read in orc.

FloatProgress(value=0.0, bar_style='info', description='Progress:', layout=Layout(height='25px', width='50%'),…

+-------+--------------------+--------+--------+-------------+--------------+----+-----+---+
|stop_id|           stop_name|stop_lat|stop_lon|location_type|parent_station|year|month|day|
+-------+--------------------+--------+--------+-------------+--------------+----+-----+---+
|8799803|Annecy, gare rout...|      46|       6|             |              |2021|    8| 18|
|1100008|Zell (Wiesental),...|      48|       8|             |              |2021|    8| 18|
|1100009|Zell (Wiesental),...|      48|       8|             |              |2021|    8| 18|
|1100010|           Atzenbach|      48|       8|             |              |2021|    8| 18|
|1100011|     Mambach, Brücke|      48|       8|             |              |2021|    8| 18|
|1100012|  Mambach, Mühlschau|      48|       8|             |              |2021|    8| 18|
|1100013|  Mambach, Silbersau|      48|       8|             |              |2021|    8| 18|
|1100014|       Fröhnd, Wühre|      48|       8|             |        

In [7]:
stop_times.show()

FloatProgress(value=0.0, bar_style='info', description='Progress:', layout=Layout(height='25px', width='50%'),…

+--------------------+------------+--------------+-----------+-------------+-----------+-------------+----+-----+---+
|             trip_id|arrival_time|departure_time|    stop_id|stop_sequence|pickup_type|drop_off_type|year|month|day|
+--------------------+------------+--------------+-----------+-------------+-----------+-------------+----+-----+---+
|231.TA.91-9-F-j22...|    06:58:00|      06:58:00|8506206:0:4|            7|          0|            0|2022|   12|  7|
|7041.TA.91-10-A-j...|    19:26:00|      19:26:00|    8500994|           20|          0|            0|2022|   12|  7|
|232.TA.91-9-F-j22...|    08:35:00|      08:35:00|8506200:0:5|            1|          0|            0|2022|   12|  7|
|7041.TA.91-10-A-j...|    19:27:00|      19:27:00|    8500079|           21|          0|            0|2022|   12|  7|
|232.TA.91-9-F-j22...|    08:37:00|      08:37:00|8506201:0:1|            2|          0|            0|2022|   12|  7|
|7041.TA.91-10-A-j...|    19:29:00|      19:29:00|    85

In [8]:
trips.show()

FloatProgress(value=0.0, bar_style='info', description='Progress:', layout=Layout(height='25px', width='50%'),…

+-------------+----------+--------------------+--------------------+---------------+------------+----+-----+---+
|     route_id|service_id|             trip_id|       trip_headsign|trip_short_name|direction_id|year|month|day|
+-------------+----------+--------------------+--------------------+---------------+------------+----+-----+---+
|91-10-A-j22-1|  TA+8i000|1.TA.91-10-A-j22-...|Oberwil BL, Hüsli...|          51125|           0|2022|   12|  7|
|91-10-A-j22-1|     TA+6V|10.TA.91-10-A-j22...|Oberwil BL, Hüsli...|          20605|           0|2022|   12|  7|
|91-10-A-j22-1|  TA+f6n00|100.TA.91-10-A-j2...|Oberwil BL, Hüsli...|          22991|           0|2022|   12|  7|
|91-10-A-j22-1|     TA+rV|1000.TA.91-10-A-j...|    Dornach, Bahnhof|          51153|           0|2022|   12|  7|
|91-10-A-j22-1|     TA+rV|1001.TA.91-10-A-j...|    Dornach, Bahnhof|          51061|           0|2022|   12|  7|
|91-10-A-j22-1|     TA+rV|1002.TA.91-10-A-j...|    Dornach, Bahnhof|          50967|           0

In [9]:
calendar.show()

FloatProgress(value=0.0, bar_style='info', description='Progress:', layout=Layout(height='25px', width='50%'),…

+----------+------+-------+---------+--------+------+--------+------+----------+--------+----+-----+---+
|service_id|monday|tuesday|wednesday|thursday|friday|saturday|sunday|start_date|end_date|year|month|day|
+----------+------+-------+---------+--------+------+--------+------+----------+--------+----+-----+---+
|        TA|  TRUE|   TRUE|     TRUE|   FALSE| FALSE|   FALSE|  TRUE|  20221211|20221214|2022|   12|  7|
|      TA#1|  TRUE|   TRUE|     TRUE|    TRUE|  TRUE|    TRUE|  TRUE|  20211212|20221210|2022|   12|  7|
|  TA+00000| FALSE|  FALSE|    FALSE|   FALSE| FALSE|    TRUE| FALSE|  20211212|20221210|2022|   12|  7|
|  TA+00010|  TRUE|   TRUE|     TRUE|   FALSE| FALSE|   FALSE|  TRUE|  20211212|20221210|2022|   12|  7|
|  TA+001c0|  TRUE|   TRUE|     TRUE|    TRUE|  TRUE|   FALSE| FALSE|  20211212|20221210|2022|   12|  7|
|  TA+00200|  TRUE|   TRUE|     TRUE|    TRUE|  TRUE|    TRUE|  TRUE|  20211212|20221210|2022|   12|  7|
|  TA+00280|  TRUE|   TRUE|     TRUE|    TRUE|  TRUE|  

In [10]:
routes.show(10)

FloatProgress(value=0.0, bar_style='info', description='Progress:', layout=Layout(height='25px', width='50%'),…

+--------------+---------+----------------+---------------+----------+----------+----+-----+---+
|      route_id|agency_id|route_short_name|route_long_name|route_desc|route_type|year|month|day|
+--------------+---------+----------------+---------------+----------+----------+----+-----+---+
|92-A00-5-j23-1|   80_BOD|               1|               |         B|       700|2022|   12|  7|
|92-A00-5-j22-1|   80_BOD|               1|               |         B|       700|2022|   12|  7|
|92-A00-4-j23-1|   80_BOD|               4|               |         B|       700|2022|   12|  7|
|92-A00-4-j22-1|      839|               1|               |       EXB|       702|2022|   12|  7|
|92-A00-3-j23-1|   80_BOD|               2|               |         B|       700|2022|   12|  7|
|92-A00-3-j22-1|     7090|               3|               |         B|       700|2022|   12|  7|
|92-A00-2-j23-1|   80_BOD|               4|               |         B|       700|2022|   12|  7|
|92-A00-2-j22-1|     7090|    

In [11]:
actual_temp = spark.read.load("/data/sbb/part_orc/istdaten", format="orc", sep=";", inferSchema="true", header="true")
#Rename the columns, from German to English
actual_condition = actual_temp.withColumnRenamed("betriebstag", "Date_of_trip")\
                .withColumnRenamed("fahrt_bezeichner", "Trip_id")\
                .withColumnRenamed("betreiber_id", "Operator_id")\
                .withColumnRenamed("betreiber_abk", "Operator_abk")\
                .withColumnRenamed("betreiber_name", "Operator_name")\
                .withColumnRenamed("produkt_id", "Transport_type")\
                .withColumnRenamed("linien_id", "Train_number(train)")\
                .withColumnRenamed("linien_text", "Service type(train)")\
                .withColumnRenamed("umlauf_id", "Circulation_id")\
                .withColumnRenamed("verkehrsmittel_text", "Means_of_transport_text")\
                .withColumnRenamed("zusatzfahrt_tf", "If_additional")\
                .withColumnRenamed("faellt_aus_tf", "If_failed")\
                .withColumnRenamed("bpuic", "Stop_id")\
                .withColumnRenamed("haltestellen_name", "Stop_name")\
                .withColumnRenamed("ankunftszeit", "Arrival_time")\
                .withColumnRenamed("an_prognose", "Actual_arrival_time")\
                .withColumnRenamed("abfahrtszeit", "Departure_time")\
                .withColumnRenamed("ab_prognose", "Actual_departure_time")\
                .withColumnRenamed("durchfahrt_tf", "Not_stop")
actual_condition.printSchema()
actual_condition.show(5)

FloatProgress(value=0.0, bar_style='info', description='Progress:', layout=Layout(height='25px', width='50%'),…

root
 |-- Date_of_trip: string (nullable = true)
 |-- Trip_id: string (nullable = true)
 |-- Operator_id: string (nullable = true)
 |-- Operator_abk: string (nullable = true)
 |-- Operator_name: string (nullable = true)
 |-- Transport_type: string (nullable = true)
 |-- Train_number(train): string (nullable = true)
 |-- Service type(train): string (nullable = true)
 |-- Circulation_id: string (nullable = true)
 |-- Means_of_transport_text: string (nullable = true)
 |-- If_additional: string (nullable = true)
 |-- If_failed: string (nullable = true)
 |-- Stop_id: string (nullable = true)
 |-- Stop_name: string (nullable = true)
 |-- Arrival_time: string (nullable = true)
 |-- Actual_arrival_time: string (nullable = true)
 |-- an_prognose_status: string (nullable = true)
 |-- Departure_time: string (nullable = true)
 |-- Actual_departure_time: string (nullable = true)
 |-- ab_prognose_status: string (nullable = true)
 |-- Not_stop: string (nullable = true)
 |-- year: integer (nullable = 

## We first transform the latitude and longtitude into distance and find all stops that are in 15km radius of Zurich HB

The **_Haversine formula_** is chosen for this task. It is a mathematical formula used to calculate the distance between two points on the Earth's surface using their latitude and longitude coordinates. It is commonly used in navigation and geographical applications.The Haversine formula takes into account the curvature of the Earth and calculates the distance between two points as the arc length along a great circle. It uses the latitude and longitude coordinates of the two points to calculate the central angle between them. The formula then computes the distance based on the central angle and the radius of the Earth.

The formula is as follows:

**d = 2 * R * arcsin(sqrt(sin^2((latitude_2 - latitude_1)/2) + cos(latitude_1) * cos(latitude_2) * sin^2((longitude_2 - longitude_1)/2)))**

Where:

d is the distance between the two points.

R is the radius of the Earth (typically in kilometers or miles).

latitude_1, longitude_1 are the latitude and longitude coordinates of the first point.

latitude_2, longitude_2 are the latitude and longitude coordinates of the second point.

By applying the Haversine formula, one can accurately calculate the distance between two points on the Earth's surface using their latitude and longitude values.

In [32]:
from math import sin, cos, sqrt, atan2, radians
import pyspark.sql.functions as F
from pyspark.sql.types import FloatType

@F.udf(returnType=FloatType())

#Implement the Haversine formula. 
def distance_calculation(latitude_1, longitude_1, latitude_2, longitude_2):
    #Use Haversine formula. 
    #The Haversine formula calculates the distance between two points on a sphere 
    #(such as the Earth) based on their latitude and longitude.
    radius_of_Earth = 6371.0 #Earth radius, just refer to the actual data 
    
    #First, Convert latitude and longitude from degrees to radians
    latitude_1 = radians(float(latitude_1))
    latitude_2 = radians(float(latitude_2))
    longitude_1 = radians(float(longitude_1))
    longitude_2 = radians(float(longitude_2))

    ## Haversine formula implementation
    delta_latitude = latitude_2 - latitude_1
    delta_longitude = longitude_2 - longitude_1

    a = cos(latitude_1)*cos(latitude_2)*sin(delta_longitude/2)**2+sin(delta_latitude/2)**2
    c = 2*atan2(sqrt(a), sqrt(1-a))

    distance = radius_of_Earth * c 
    return distance

stops_in_15 = stops_csv.where(distance_calculation(F.lit(47.378177), F.lit(8.540192), F.col("stop_lat"), F.col("stop_lon")) <=15)

print("There are " + str(stops_in_15.distinct().count()) + " stops that are in 15km radius of Zurich HB")

stops_in_15.show()

FloatProgress(value=0.0, bar_style='info', description='Progress:', layout=Layout(height='25px', width='50%'),…

There are 538414 stops that are in 15km radius of Zurich HB
+-------------+--------------------+----------------+----------------+-------------+--------------+----+-----+---+
|      stop_id|           stop_name|        stop_lat|        stop_lon|location_type|parent_station|year|month|day|
+-------------+--------------------+----------------+----------------+-------------+--------------+----+-----+---+
|      8500926|Oetwil a.d.L., Sc...|47.4236270123012| 8.4031825286317|         null|          null|2022|   12|  7|
|      8502186|Dietikon Stoffelbach|47.3933267759652|8.39896044679575|         null| Parent8502186|2022|   12|  7|
|8502186:0:1/2|Dietikon Stoffelbach|47.3933997509195|8.39894248049007|         null| Parent8502186|2022|   12|  7|
|      8502187|Rudolfstetten Hof...|47.3646702178563|8.37695172233176|         null| Parent8502187|2022|   12|  7|
|8502187:0:1/2|Rudolfstetten Hof...|47.3647371479356|8.37703257070734|         null| Parent8502187|2022|   12|  7|
|      8502188|   Zu

## We then see the walking distance.

In [34]:
walking_df = stops_in_15.select(F.col("stop_id").alias("stop_id_1"), F.col("stop_name").alias("stop_name_1"), F.col("stop_lat").alias("stop_lat_1"), F.col("stop_lon").alias("stop_lon_1")) \
    .crossJoin(stops_in_15.select(F.col("stop_id").alias("stop_id_2"), F.col("stop_name").alias("stop_name_2"), F.col("stop_lat").alias("stop_lat_2"),F.col("stop_lon").alias("stop_lon_2"))) \
    .withColumn("distance", lat_long_dist(F.col("stop_lat_1"), F.col("stop_lon_1"), F.col("stop_lat_2"), F.col("stop_lon_2"))) \
    .select(F.col("stop_id_1"), F.col("stop_name_1"), F.col("stop_id_2"), F.col("stop_name_2"), F.col("distance")) \
    .filter("distance<=0.5 and distance>0.0")

walking_df = walking_df.withColumn("used_time", walking_df.distance*1200).select("stop_id_1","stop_name_1","stop_id_2","stop_name_2","used_time")

FloatProgress(value=0.0, bar_style='info', description='Progress:', layout=Layout(height='25px', width='50%'),…

## According to the requirement of the task, we select only the weekdays.

The weekdays filtering can be done in calendar and then we use service_id to join other dataframes to get the transportation methods in weekdays.

In [37]:
weekdays_only = calendar.where("monday = 1 and tuesday = 1 and wednesday = 1 and thursday = 1 and friday = 1").select('service_id').distinct()

FloatProgress(value=0.0, bar_style='info', description='Progress:', layout=Layout(height='25px', width='50%'),…

## We then implement all the joins using primary keys

In [43]:
trips_in_weekdays = trips.join(weekdays_only, "service_id")

trips_routes_in_weekdays = trips_in_weekdays.join(routes, "route_id")

stop_times_trips_routes_in_weekdays = stop_times.join(trips_routes_in_weekdays, "trip_id")

final_df_filtered_and_combined = stop_times_trips_routes_in_weekdays.join(stops_in_15, "stop_id")
#trips_routes_in_weekdays.show()

FloatProgress(value=0.0, bar_style='info', description='Progress:', layout=Layout(height='25px', width='50%'),…